In [2883]:
import pandas as pd
import re
import datetime

from disaster_extr_constants import *
from disaster_extr_helpers import *

In [2884]:
YEAR = 2020
disaster_type = 'heat_wave'
file_path = 'data/'+str(YEAR)+'_'+disaster_type+'_climate_processed_csv.bz2'

In [2885]:
data = 'data/emdat_processed.csv'
parse_dates = ['StartDate', 'EndDate']
df_emdat = pd.read_csv(data, index_col="Dis No", parse_dates = parse_dates)

In [2886]:
disaster_df = pd.read_csv(file_path, parse_dates = ['date'], compression='bz2')

In [2887]:
df_heat_wave = get_df_disaster(df_emdat, HEAT_WAVES, HEAT_WAVES_val)
df_heat_wave_bounds = retrieve_bounding_dates(df_heat_wave)

df_heat_wave = get_df_disaster(df_emdat, HEAT_WAVES_2020, HEAT_WAVES_2020_val)
df_heat_wave_bounds = retrieve_bounding_dates(df_heat_wave)

disaster_df_bounds = df_heat_wave_bounds

start_YEAR, end_YEAR = disaster_df_bounds.loc[YEAR-1].MinStartDate, disaster_df_bounds.loc[YEAR-1].MaxEndDate

In [2888]:
df_heat_wave_bounds

,MinStartDate,MaxEndDate
Year,,
2019,2019-09-01,2020-02-01


In [2889]:
start_YEAR, end_YEAR

(Timestamp('2019-09-01 00:00:00'), Timestamp('2020-02-01 00:00:00'))

In [2890]:
one_month = datetime.timedelta(days=31)

if end_YEAR - start_YEAR < one_month:
    # Add 10 days to end date
    ten_days = datetime.timedelta(days=10)
    end_YEAR += ten_days

In [2891]:
start_YEAR, end_YEAR

(Timestamp('2019-09-01 00:00:00'), Timestamp('2020-02-01 00:00:00'))

In [2892]:
start_YEAR, end_YEAR = start_YEAR.strftime("%Y-%m-%d"), end_YEAR.strftime("%Y-%m-%d")

In [2893]:
start_YEAR, end_YEAR

('2019-09-01', '2020-02-01')

In [2894]:
df_disaster_start_end = df_time_interval(disaster_df, start_YEAR, end_YEAR,date_attr='date')

In [2895]:
df_disaster_start_end

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
0,2019-10-30-054451,It is more likely these catalysts will be impl...,Rose Amal,['Q25513876'],2019-10-30 09:58:11,1,"[['Rose Amal', '0.9587'], ['None', '0.0413']]",['https://www.miragenews.com/chemical-engineer...,E
1,2019-09-13-002852,acutely aware we are in the midst of a climate...,Michael Thomson,"['Q16239395', 'Q23418728', 'Q6834872', 'Q68348...",2019-09-13 00:00:00,1,"[['Michael Thomson', '0.6821'], ['None', '0.25...",['http://theherald.com.au/story/6383605/univer...,E
2,2019-09-25-005329,and bear varying degrees of culpability.,None,[],2019-09-25 22:44:24,3,"[['None', '0.6772'], ['Ron DeSantis', '0.2969'...",['http://sunshinestatenews.com/story/special-m...,E
3,2019-09-26-057277,It's managing those zillions of microbial cell...,Thomas Crowther,['Q22958878'],2019-09-26 11:00:45,1,"[['Thomas Crowther', '0.9631'], ['None', '0.03...",['https://www.yaleclimateconnections.org/2019/...,E
4,2019-10-12-030285,I've spoken to people and they didn't realise ...,None,[],2019-10-12 06:03:50,1,"[['None', '0.557'], ['John Yates', '0.443']]",['https://www.horseandhound.co.uk/news/blister...,E
...,...,...,...,...,...,...,...,...,...
121381,2020-01-21-033625,I think we are all sad to see the consequences...,Toni Nadal,['Q2118577'],2020-01-21 05:05:58,1,"[['Toni Nadal', '0.844'], ['None', '0.156']]",['https://www.foxsportsasia.com/tennis/austral...,E
121382,2020-01-20-037617,It's also an event where disqualifications can...,None,[],2020-01-20 00:14:03,2,"[['None', '0.5625'], ['Quentin Rew', '0.4375']]","['http://home.nzcity.co.nz/nl.aspx?4427659', '...",E
121383,2020-01-05-026011,"On the one hand, you've got ta see it, but, on...",Roger Deakins,['Q460277'],2020-01-05 17:00:49,1,"[['Roger Deakins', '0.7504'], ['None', '0.2496']]",['https://www.indiewire.com/2020/01/1917-cinem...,E
121384,2020-01-24-095416,Unbelievable 1440 degrees jump by Kelly Sildar...,Taavi Rõivas,['Q3785077'],2020-01-24 00:00:00,2,"[['Taavi Rõivas', '0.8461'], ['Kelly Sildaru',...",['https://www.forbes.com/sites/michellebruton/...,E


In [3054]:
heat_tags_2020_pos = pd.DataFrame(
    {'tags': [
        r'\b([bB]ush [fF]ires?)\b',
        r'(?=.*\b(Australia)\b)(?=.*\b([tT]emperatures?)\b)',
        r'(?=.*\b(Australia)\b)(?=.*\b([fF]ires?)\b)',
        r'(?=.*\b(Australia)\b)(?=.*\b([hH]eat[ -]?[wW]aves?)\b)',
        r'(?=.*\b(Sydney|New South Wales|Melbourne)\b)(?=.*\b([hH]eat[ -]?[wW]aves?)\b)'
    ]
    }
)

In [3055]:
tags_pos_list = heat_tags_2020_pos.tags.values.tolist()

In [3056]:
tags_pos_list

['\\b([bB]ush [fF]ires?)\\b',
 '(?=.*\\b(Australia)\\b)(?=.*\\b([tT]emperatures?)\\b)',
 '(?=.*\\b(Australia)\\b)(?=.*\\b([fF]ires?)\\b)',
 '(?=.*\\b(Australia)\\b)(?=.*\\b([hH]eat[ -]?[wW]aves?)\\b)',
 '(?=.*\\b(Sydney|New South Wales|Melbourne)\\b)(?=.*\\b([hH]eat[ -]?[wW]aves?)\\b)']

In [3057]:
# Return regex pattern
regex_pattern_pos = r'|'.join(tags_pos_list)

In [3058]:
filtered_for_pos_new = extract_quotes(df_disaster_start_end, regex_pattern_pos).sort_values(by='date')

In [3060]:
filtered_for_pos_new

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
22862,2019-09-01-042264,The large air tankers we contract throughout t...,James Morris,"['Q1680858', 'Q17612656', 'Q23617803', 'Q27477...",2019-09-01 06:54:00,1,"[['James Morris', '0.9701'], ['None', '0.0299']]",['http://www.bordermail.com.au/story/6360081/h...,E
89996,2019-09-02-012365,"First of all, I want to say that I have huge r...",Daniil Medvedev,['Q21622022'],2019-09-02 14:30:56,1,"[['Daniil Medvedev', '0.8972'], ['None', '0.07...",['https://www.tennisworldusa.org/tennis/news/N...,E
3371,2019-09-02-070225,We urge all residents in the MIA District to b...,Kevin Adams,"['Q28919659', 'Q57440397', 'Q6395700']",2019-09-02 21:00:00,2,"[['Kevin Adams', '0.9564'], ['None', '0.0437']]",['http://areanews.com.au/story/6363273/hanwood...,E
61360,2019-09-03-076962,The demand for meat in Australia and around th...,Emily Rice,['Q27736836'],2019-09-03 23:32:34,1,"[['Emily Rice', '0.8845'], ['None', '0.1155']]",['http://www.adnews.com.au/news/peta-blames-me...,E
49751,2019-09-04-019958,Everyone should have a bush fire survival plan...,Kevin Adams,"['Q28919659', 'Q57440397', 'Q6395700']",2019-09-04 03:00:00,1,"[['Kevin Adams', '0.9795'], ['None', '0.0205']]",['http://www.irrigator.com.au/story/6366306/da...,E
...,...,...,...,...,...,...,...,...,...
111067,2020-01-31-102308,Tomorrow will be the peak of the heatwave in N...,Andrew Barr,"['Q16204680', 'Q19324999', 'Q4756289', 'Q56418...",2020-01-31 16:25:09,1,"[['Andrew Barr', '0.5493'], ['None', '0.4507']]",['http://taipeitimes.com/News/front/archives/2...,E
109555,2020-01-31-047190,In the catastrophic fires here (which were ena...,Michael Mann,"['Q1425193', 'Q1827335', 'Q1928511', 'Q270097'...",2020-01-31 18:05:36,1,"[['Michael Mann', '0.8644'], ['None', '0.1356']]",['http://www.adn.com/nation-world/2020/01/31/w...,E
116304,2020-01-31-102307,Tomorrow will be the peak of the heatwave in N...,None,[],2020-01-31 21:00:00,1,"[['None', '0.5818'], ['Andrew Barr', '0.4182']]",['http://www.straitstimes.com/asia/australianz...,E
114401,2020-01-31-064632,New Zealand's highest temperature's don't ofte...,Philip Duncan,"['Q3378766', 'Q7183478']",2020-01-31 21:07:31,1,"[['Philip Duncan', '0.8995'], ['None', '0.1005']]",['http://www.voxy.co.nz/national/5/357453'],E


In [3047]:
extract_quotes(filtered_for_pos, r'\b([bB]ush [fF]ires?)\b', complement=True).iloc[0].quotation

"First of all, I want to say that I have huge respect for both of them. They are both amazing champions of our sport. One a little bit more of champion than another, but still... When I was playing junior Grand Slams, it was, like, a classic semifinal, Djokovic, Wawrinka, I remember two times in Australia, I think, and final of Roland Garros. It was all when I was a junior. Some amazing tennis from both of them, like, out of this world. I will probably watch the match and enjoy it. Talking about them differently, I mean, playing Novak is always tough, and especially five-setters. At this moment we played two times out of five sets. I lost both of them. Our games are really physical. Even he said it. It's not going to be easy. Or if Stan wins, I played him in Wimbledon. I think that was our only meeting. Totally different. Grass, I was young, I was less experienced, so I was on fire that match. I think that Stan also in Grand Slams, he's playing right now better than on other tournament

In [3067]:
heat_tags_2020_neg = pd.DataFrame(
    {'tags': [
        r'\b([tT]ongues?)\b',
        r'\b([tT]urkey)\b',
        r'\b([gG]ames?)\b',
        r'\b([pP]layers?)\b',
        r'\b([sS]tudios?)\b',
        r'\b([hH]elmets?)\b',
        r'\b(Arkansas)\b',
        r'\b([vV]iolences?)\b',
        r'\b([fF]all)\b',
        r'\b(Chicago(ans?)?)\b',
        r'\b([cC]redits?)\b',
        r'\b([gG]angs?)\b',
        r'\b([sS]ales?)\b',
        r'\b([hH]ousing)\b',
        r'\b([bB]lokes?)\b',
        r'\b([sS]hadows?)\b',
        r'\b([hH]oneys?)\b',
        r'\b([tT]obacco)\b',
        r'\b([lL]aps?)\b',
        r'\b([pP]aris ([cC]limate ([cC]hange )?)?[aA]greements?)\b',
        r'\b([pP]aris (([gG]lobal )?[cC]limate )?[aA]ccords?)\b',
        r'\b(Missouri)\b',
        r'\b(wartime)\b',
        r'\b([dD]inners?)\b',
        r'\b([iI]ndustr(y|ies))\b',
        r'\b(Boston)\b',
        r'\b(EU)\b',
        r'\b(Canada)\b',
        r'\b(United States|U.?S.?)\b',
        r'\b([bB]ananas?)\b'
        r'\b([sS]treets?)\b',
        r'\b([tT]ackles?)\b',
        r'\b([qQ]uickly)\b',
        r'\b([pP]reventable)\b',
        r'\b([cC]andidates?)\b',
        r'\b([mM]usic)\b',
        r'\b([rR]omances?)\b',
        r'\b([mM]arines?)\b',
        r'\b([tT]urns? up)\b',
        r'\b([pP]lay(ers?)?)\b',
        r'\b(Pirelli)\b',
        r'\b(Mercedes-Benz)\b',
        r'\b([cC]lassrooms?)\b',
        r'\b(New York(ers?)?)\b',
        r'\b(([tT]wo|[oO]ne) degrees?)\b',
        r'\b([mM]inus|[nN]egative)\b',
        r'\b([sS]hops?)\b',
        r'\b(MTA)\b',
        r'\b(Europe)\b',
        r'\b(London)\b',
        r'\b(Paris)\b',
        r'\b(Belgium)\b',
        r'\b(France)\b',
        r'\b(Germany)\b',
        r'\b(Luxembourg)\b',
        r'\b(Netherlands)\b',
        r'\b(England)\b',
        r'\b([mM][ée]t[ée]o [fF]rance)\b',
        r'\b([gG]allargues-le-[mM]ontueux)\b',
        r'\b([bB]erlin[- ][tT]empelhof)\b',
        r'\b([bB]randenburg)\b',

        

        r'\b([rR]eactors?)\b',
        r'\b([cC]lients?)\b',
        r'\b([rR]ock)\b',
        r'\b([bB]eers?)\b',
        r'\b([lL]ovely)\b',
        r'\b([aA]dministrations?)\b',
        r'\b([pP]ets?)\b',
        r'\b([jJ]ew(s|ish))\b',
        r'\b([mM]edical(ly)?)\b',
        r'\b([wW]arm[- ]?ups?)\b',
        r'\b([hH]omeless(ness)?)\b',
        r'\b(New[- ]?England)\b',
        r'\b([dD]oors?)\b',
        r'\b([bB]at(sm[ea]n)?)\b',
        r'\b([hH]and warmers?)\b',
        r'\b([fF]reezing)\b',
        r'\b(Japan)\b',
        r'\b((1|1\.5|2|2\.5) degrees?)\b',
        r'\b(20(20|30|40))\b',
        r'\b(Mars)\b',
        r'\b([cC]oldest)\b',
        r'\b([fF]rozen?)\b',
        r'\b([hH]umid(ity)?)\b',
        r'\b([vV]olcano(es?)?)\b',
        r'\b(corridor)\b',
        r'\b([cC]ores?)\b',
        r'\b([lL]ow temperatures?)\b',
        r'\b([iI]nternal)\b',
        r'\b([iI]nsane)\b',
        r'\b([aA]thletes?)\b',

        


        r'\b([rR]osters?)\b',
        r'\b([sS]tud(y|ies))\b',
        r'\b(America(ns?)?)\b',
        r'\b([rR]ules?)\b',
        r'\b([sS]ediments?)\b',
        r'\b([sS]eriously)\b',
        r'\b([tT]heor(y|etical))\b',
        r'\b([eE]xperiment(s|al|ally)?)\b',
        r'\b([hH]eat cramps?)\b',
        r'\b([pP]izzas?)\b',
        r'\b([rR]ails?)\b',
        r'\b(NYPD)\b',
        r'\b([dD]roughts?)\b',

    ]
    }
)

In [2997]:
tag_neg_list = heat_tags_2020_neg.tags.values.tolist()

# Return regex pattern
regex_pattern_neg = r'|'.join(tag_neg_list)

In [3068]:
filtered_for_pos_then_neg_new = extract_quotes(filtered_for_pos_new, regex_pattern_neg, complement=True)

In [3070]:
len(filtered_for_pos_then_neg_new)

961

In [3069]:
len(filtered_for_pos_then_neg)

956

In [3000]:
filtered_for_pos_then_neg

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
3371,2019-09-02-070225,We urge all residents in the MIA District to b...,Kevin Adams,"['Q28919659', 'Q57440397', 'Q6395700']",2019-09-02 21:00:00,2,"[['Kevin Adams', '0.9564'], ['None', '0.0437']]",['http://areanews.com.au/story/6363273/hanwood...,E
61360,2019-09-03-076962,The demand for meat in Australia and around th...,Emily Rice,['Q27736836'],2019-09-03 23:32:34,1,"[['Emily Rice', '0.8845'], ['None', '0.1155']]",['http://www.adnews.com.au/news/peta-blames-me...,E
49751,2019-09-04-019958,Everyone should have a bush fire survival plan...,Kevin Adams,"['Q28919659', 'Q57440397', 'Q6395700']",2019-09-04 03:00:00,1,"[['Kevin Adams', '0.9795'], ['None', '0.0205']]",['http://www.irrigator.com.au/story/6366306/da...,E
53818,2019-09-04-023275,"Given the undeniable increase in frequency, in...",Ollie Jay,['Q45993182'],2019-09-04 11:46:13,1,"[['Ollie Jay', '0.8418'], ['None', '0.1582']]",['https://www.miragenews.com/climate-increasin...,E
82022,2019-09-07-045579,"The peat bush fire incident in Kuala Baram, Mi...",Wan Azizah Wan Ismail,['Q468579'],2019-09-07 07:40:00,3,"[['Wan Azizah Wan Ismail', '0.9043'], ['None',...",['http://www.bernama.com/en/news.php?id=176507...,E
...,...,...,...,...,...,...,...,...,...
108180,2020-01-31-068312,Only in rainy weather does the Australian bush...,Les Murray,"['Q259841', 'Q6529770']",2020-01-31 13:00:00,1,"[['Les Murray', '0.8092'], ['None', '0.1249'],...",['http://www.canberratimes.com.au/story/660389...,E
112520,2020-01-31-094312,There's now no higher priority for the ACT gov...,Andrew Barr,"['Q16204680', 'Q19324999', 'Q4756289', 'Q56418...",2020-01-31 14:14:52,2,"[['Andrew Barr', '0.6281'], ['None', '0.3719']]",['http://www.blackpoolgazette.co.uk/news/read-...,E
97960,2020-01-31-081583,The ACT is now facing the worst bush fire thre...,Andrew Barr,"['Q16204680', 'Q19324999', 'Q4756289', 'Q56418...",2020-01-31 14:14:52,2,"[['Andrew Barr', '0.9644'], ['None', '0.0356']]",['http://www.blackpoolgazette.co.uk/news/read-...,E
109555,2020-01-31-047190,In the catastrophic fires here (which were ena...,Michael Mann,"['Q1425193', 'Q1827335', 'Q1928511', 'Q270097'...",2020-01-31 18:05:36,1,"[['Michael Mann', '0.8644'], ['None', '0.1356']]",['http://www.adn.com/nation-world/2020/01/31/w...,E


In [3001]:
filtered_for_pos_then_neg_TEST = extract_quotes(
    filtered_for_pos_then_neg, 
    r'something', 
    field='urls', 
    complement=True)

In [3002]:
len(filtered_for_pos_then_neg_TEST)

894

In [3035]:
filtered_for_pos_then_neg

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
3371,2019-09-02-070225,We urge all residents in the MIA District to b...,Kevin Adams,"['Q28919659', 'Q57440397', 'Q6395700']",2019-09-02 21:00:00,2,"[['Kevin Adams', '0.9564'], ['None', '0.0437']]",['http://areanews.com.au/story/6363273/hanwood...,E
61360,2019-09-03-076962,The demand for meat in Australia and around th...,Emily Rice,['Q27736836'],2019-09-03 23:32:34,1,"[['Emily Rice', '0.8845'], ['None', '0.1155']]",['http://www.adnews.com.au/news/peta-blames-me...,E
49751,2019-09-04-019958,Everyone should have a bush fire survival plan...,Kevin Adams,"['Q28919659', 'Q57440397', 'Q6395700']",2019-09-04 03:00:00,1,"[['Kevin Adams', '0.9795'], ['None', '0.0205']]",['http://www.irrigator.com.au/story/6366306/da...,E
53818,2019-09-04-023275,"Given the undeniable increase in frequency, in...",Ollie Jay,['Q45993182'],2019-09-04 11:46:13,1,"[['Ollie Jay', '0.8418'], ['None', '0.1582']]",['https://www.miragenews.com/climate-increasin...,E
82022,2019-09-07-045579,"The peat bush fire incident in Kuala Baram, Mi...",Wan Azizah Wan Ismail,['Q468579'],2019-09-07 07:40:00,3,"[['Wan Azizah Wan Ismail', '0.9043'], ['None',...",['http://www.bernama.com/en/news.php?id=176507...,E
...,...,...,...,...,...,...,...,...,...
108180,2020-01-31-068312,Only in rainy weather does the Australian bush...,Les Murray,"['Q259841', 'Q6529770']",2020-01-31 13:00:00,1,"[['Les Murray', '0.8092'], ['None', '0.1249'],...",['http://www.canberratimes.com.au/story/660389...,E
112520,2020-01-31-094312,There's now no higher priority for the ACT gov...,Andrew Barr,"['Q16204680', 'Q19324999', 'Q4756289', 'Q56418...",2020-01-31 14:14:52,2,"[['Andrew Barr', '0.6281'], ['None', '0.3719']]",['http://www.blackpoolgazette.co.uk/news/read-...,E
97960,2020-01-31-081583,The ACT is now facing the worst bush fire thre...,Andrew Barr,"['Q16204680', 'Q19324999', 'Q4756289', 'Q56418...",2020-01-31 14:14:52,2,"[['Andrew Barr', '0.9644'], ['None', '0.0356']]",['http://www.blackpoolgazette.co.uk/news/read-...,E
109555,2020-01-31-047190,In the catastrophic fires here (which were ena...,Michael Mann,"['Q1425193', 'Q1827335', 'Q1928511', 'Q270097'...",2020-01-31 18:05:36,1,"[['Michael Mann', '0.8644'], ['None', '0.1356']]",['http://www.adn.com/nation-world/2020/01/31/w...,E


In [3071]:
extract_quotes(filtered_for_pos_then_neg_new, r'(?=.*\b(Sydney|New South Wales|Melbourne)\b)(?=.*\b([hH]eat[ -]?[wW]aves?)\b)', complement=False).quotation.values

array(["From today we're expecting what we call a moderate intensity heatwave, and that does increase across from the Wednesday through to Friday period with some pockets of severe being reached across inland New South Wales,",
       "It's such an enormously big fire, it's beyond human ability to control. Heatwave looming With a severe heatwave forecast to hit NSW towards the end of the week -- including temperatures tipping over 45 in Sydney's west on Saturday -- Ms Burford said firefighters would continue to make use of relatively mild conditions in the couple of days they have to backburn before the weather deteriorates. Saturday's extreme heat is shaping up to be the big challenge, she said.",
       "A record heatwave is developing across southern Australia. We're expecting to break temperature records at dozens of locations across South Australia, the ACT, Victoria and New South Wales,",
       'The project, called Heat Smart Western Sydney, will bring together local councils, g

In [3033]:
extract_quotes(filtered_for_pos_then_neg, r'county', field='urls', complement=False).quotation.values

array(["The problem is that their fire seasons have become a lot longer as well, so we're being left vulnerable by not having those types of aircraft available to us in Australia at a time when we most need them.",
       'Our main concern is with bush fire that our fire seasons are becoming much, much longer that they used to be,',
       'We continue to be shocked and deeply saddened to hear about the fires that are destroying homes, livelihoods, and wildlife across much of Australia,',
       'Our friend Greg Page suffered a cardiac arrest at the end of the bush fire relief performance and was taken to hospital,',
       'Our heartfelt condolences go out to the family, friends and colleagues of the crew from Tanker 134 that wre lost during a tragic crash while battling the devastating Australia fires. Cal Fire stands with you during this difficult time,'],
      dtype=object)

In [1295]:
def extract_quotes_protected(df, protected_regex, unwanted_regex, field='quotation', with_url=False):
    mask_protected = df[field].str.contains(protected_regex)
    df_protected_index = set(df[mask_protected].index)

    mask_unwanted = df[field].str.contains(unwanted_regex)
    df_unwanted_index = set(df[mask_unwanted].index)
    
    if with_url:
        url_regex, url_field = with_url[0], with_url[1]
        mask_protected_url = df[url_field].str.contains(url_regex)
        df_protected_url_index = set(df[mask_protected_url].index)
        df_protected_index.update(df_protected_url_index)
        
        if len(with_url) > 2:
            url_regex, url_field = with_url[2], with_url[3]
            mask_unwanted_url = df[url_field].str.contains(url_regex)
            df_unwanted_url_index = set(df[mask_unwanted_url].index)
            df_unwanted_index.update(df_unwanted_url_index)   
        
    
    resulting_inices = list(df_unwanted_index - df_protected_index)
    return df.drop(resulting_inices)

In [1537]:
filtered_extra = extract_quotes_protected(
    filtered_for_pos_then_neg, 
    r'\b([wW]orld)\b', 
    r'\b([vV]ehicles?)\b',
    with_url=[r'\.co\.uk', 'urls', r'http', 'urls']
)

/var/folders/80/18t8jx716vj1gh_49bk904tc0000gn/T/ipykernel_61644/1572180232.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  mask_protected = df[field].str.contains(protected_regex)
/var/folders/80/18t8jx716vj1gh_49bk904tc0000gn/T/ipykernel_61644/1572180232.py:5: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  mask_unwanted = df[field].str.contains(unwanted_regex)


In [1538]:
len(filtered_extra)

66

In [1539]:
filtered_extra

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
673,2019-07-19-001526,A three-day heat wave in the most northerly pl...,Dave Phillips,"['Q25190442', 'Q508561', 'Q5229572', 'Q5229573']",2019-07-19 08:00:58,2,"[['Dave Phillips', '0.8347'], ['None', '0.1653']]",['http://cbc.ca/news/technology/how-a-heat-dom...,E
18019,2019-07-19-005316,As we head into next week we are expecting som...,Dan Harris,"['Q16225310', 'Q18977889', 'Q2556156', 'Q52136...",2019-07-19 11:32:03,1,"[['Dan Harris', '0.9684'], ['None', '0.0316']]",['http://www.entertainmentdaily.co.uk/news/wea...,E
2408,2019-07-19-051686,Temperatures could reach 34C by Wednesday in p...,Andy Page,['Q4761123'],2019-07-19 12:27:00,17,"[['Andy Page', '0.9056'], ['None', '0.0944']]",['http://news.sky.com/story/uk-weather-school-...,E
15227,2019-07-19-063999,This is a heatwave coming up these next days. ...,Bill de Blasio,['Q4911497'],2019-07-19 20:37:42,3,"[['Bill de Blasio', '0.646'], ['None', '0.354']]",['http://news.bbc.co.uk/news/world-us-canada-4...,E
6701,2019-07-19-062288,They could reach 34 ° C by Wednesday in parts ...,Andy Page,['Q4761123'],2019-07-19 21:00:07,1,"[['Andy Page', '0.9458'], ['None', '0.0487'], ...",['https://www.mirror.co.uk/news/uk-news/motorw...,E
...,...,...,...,...,...,...,...,...,...
5357,2019-08-02-000590,2019 has already seen some unusual sightings a...,Neil Lucas,['Q11790488'],2019-08-02 12:01:23,1,"[['Neil Lucas', '0.9187'], ['None', '0.0813']]",['http://www.bournemouthecho.co.uk/news/178122...,E
16985,2019-08-02-001994,"A return period of 30 years like in Cambridge,...",Geert Jan van Oldenborgh,['Q58065756'],2019-08-02 14:46:00,1,"[['Geert Jan van Oldenborgh', '0.8109'], ['Non...",['https://www.telegraph.co.uk/science/2019/08/...,E
12748,2019-08-05-023326,I also want to build a stronger alliance to up...,Dominic Raab,['Q268584'],2019-08-05 00:00:00,10,"[['Dominic Raab', '0.4845'], ['None', '0.2496'...",['http://uk.investing.com/news/politics/raab-s...,E
5410,2019-08-05-047882,"like all the best religions, fear of climate c...",Boris Johnson,['Q180589'],2019-08-05 09:13:18,1,"[['Boris Johnson', '0.6236'], ['None', '0.3764']]",['http://sheffieldtelegraph.co.uk/news/opinion...,E


In [1338]:
extract_quotes(filtered_extra, r'\b([tT]okyo)\b').quotation.values

/Users/gil/EPFL/4 - Fourth Year/Semester 1/ADA/Project/ada-2021-project-data-alchemy/disaster_extr_helpers.py:17: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  mask = df[field].str.contains(regex)


array(["Athletes, coaches, sports -- If you don't think heat is going to be a factor for Tokyo 2020 -- think again! Japan is currently in a heatwave natural disaster -- with temperatures over 40 degrees celsius and high humidity. Do you have a heat plan?"],
      dtype=object)

In [1339]:
extract_quotes(filtered_extra, r'Tok', field='urls').quotation.values

array([], dtype=object)

In [3037]:
print(YEAR)

2020


In [3072]:
output_df = filtered_for_pos_then_neg_new

In [3073]:
output_df

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
3371,2019-09-02-070225,We urge all residents in the MIA District to b...,Kevin Adams,"['Q28919659', 'Q57440397', 'Q6395700']",2019-09-02 21:00:00,2,"[['Kevin Adams', '0.9564'], ['None', '0.0437']]",['http://areanews.com.au/story/6363273/hanwood...,E
61360,2019-09-03-076962,The demand for meat in Australia and around th...,Emily Rice,['Q27736836'],2019-09-03 23:32:34,1,"[['Emily Rice', '0.8845'], ['None', '0.1155']]",['http://www.adnews.com.au/news/peta-blames-me...,E
49751,2019-09-04-019958,Everyone should have a bush fire survival plan...,Kevin Adams,"['Q28919659', 'Q57440397', 'Q6395700']",2019-09-04 03:00:00,1,"[['Kevin Adams', '0.9795'], ['None', '0.0205']]",['http://www.irrigator.com.au/story/6366306/da...,E
53818,2019-09-04-023275,"Given the undeniable increase in frequency, in...",Ollie Jay,['Q45993182'],2019-09-04 11:46:13,1,"[['Ollie Jay', '0.8418'], ['None', '0.1582']]",['https://www.miragenews.com/climate-increasin...,E
82022,2019-09-07-045579,"The peat bush fire incident in Kuala Baram, Mi...",Wan Azizah Wan Ismail,['Q468579'],2019-09-07 07:40:00,3,"[['Wan Azizah Wan Ismail', '0.9043'], ['None',...",['http://www.bernama.com/en/news.php?id=176507...,E
...,...,...,...,...,...,...,...,...,...
112520,2020-01-31-094312,There's now no higher priority for the ACT gov...,Andrew Barr,"['Q16204680', 'Q19324999', 'Q4756289', 'Q56418...",2020-01-31 14:14:52,2,"[['Andrew Barr', '0.6281'], ['None', '0.3719']]",['http://www.blackpoolgazette.co.uk/news/read-...,E
111067,2020-01-31-102308,Tomorrow will be the peak of the heatwave in N...,Andrew Barr,"['Q16204680', 'Q19324999', 'Q4756289', 'Q56418...",2020-01-31 16:25:09,1,"[['Andrew Barr', '0.5493'], ['None', '0.4507']]",['http://taipeitimes.com/News/front/archives/2...,E
109555,2020-01-31-047190,In the catastrophic fires here (which were ena...,Michael Mann,"['Q1425193', 'Q1827335', 'Q1928511', 'Q270097'...",2020-01-31 18:05:36,1,"[['Michael Mann', '0.8644'], ['None', '0.1356']]",['http://www.adn.com/nation-world/2020/01/31/w...,E
116304,2020-01-31-102307,Tomorrow will be the peak of the heatwave in N...,None,[],2020-01-31 21:00:00,1,"[['None', '0.5818'], ['Andrew Barr', '0.4182']]",['http://www.straitstimes.com/asia/australianz...,E


In [3074]:
write_df_to_disk(output_df, disaster_type, YEAR, additional_text='filtered')

In [957]:
df_fire

array(['My plea to residents and visitors when outdoors is that they take great care to prevent fires, particularly as this heat wave continues.',
       "Wildfires and extreme weather are more destructive than ever and that's why we must take decisive action to protect the lives and property of the people of California. Tackling this challenge requires all of us to roll up our sleeves and to work together,",
       "Heat waves are increasing and increasing in intensity. We will have more cooling demand requiring more electricity. But if our nation continues to rely on coal-fired power plants for some of our electricity, each time we turn on the air conditioning we'll be fouling the air, causing more sickness and even deaths.",
       "Wildfires and extreme weather are more destructive than ever and that's why we must take decisive action to protect the lives and property of the people of California,",
       'Unfortunately we are also attending a number of wildfires in other areas of 